In [ ]:
import re
import json
import spacy
import unicodedata
import pandas as pd
from datasets import load_dataset
from nltk.stem import SnowballStemmer
import openpyxl

In [ ]:
ds_cleaned = load_dataset("json", data_files="wildchat_en_cleaned.jsonl", split="train")

def extract_prompt_response_pairs(conversation):
    """
    Yields (user_prompt, ai_response) pairs.
    Each pair consists of a user's message and the *next* assistant response.
    """
    pairs = []
    i = 0
    while i < len(conversation) - 1:
        user = conversation[i]
        assistant = conversation[i + 1]
        if user.get('role') == 'user' and assistant.get('role') == 'assistant':
            pairs.append({
                "prompt": user.get("content", ""),
                "response": assistant.get("content", ""),
                "prompt_clean": user.get("content_clean", ""),       
                "response_clean": assistant.get("content_clean", "") 
            })
        i += 1
    return pairs

all_pairs = []
for ex in ds_cleaned:
    for pair in extract_prompt_response_pairs(ex["conversation"]):
        pair["conversation_id"] = ex.get("conversation_id")
        pair["model"] = ex.get("model")
        all_pairs.append(pair)

with open("wildchat_prompt_response_pairs.jsonl", "w", encoding="utf-8") as f:
    for pair in all_pairs:
        f.write(json.dumps(pair, ensure_ascii=False) + "\n")
